In [ ]:
#IMPORT PYTHON PACKAGES
import seaborn as sns
import pandas as pd
import scipy as sp
import scipy.integrate
import numpy  as np
import copy
import matplotlib.pylab as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
import time
import datetime
import subprocess
import itertools
from itertools import groupby, repeat, islice
import math as math
import decimal as d
from decimal import *


#Plot settings
sns.set_style("whitegrid", rc={"axes.edgecolor": "k", "axes.linewidth":2.})

sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})


sns.set_context("notebook",rc={"grid.linewidth": 0, 
                            "font.family":"Helvetica", "axes.labelsize":24.,"xtick.labelsize":24., 
                            "ytick.labelsize":24., "legend.fontsize":18.})

color_blind_safe = sns.color_palette("colorblind", 20) 

#colors = color_blind_safe

import colorcet as cc
colors = sns.color_palette(cc.glasbey, n_colors=25)

In [ ]:
filename = ['./profile_geE1000r125', 'profile_geE1000r125_nbin5x', 'profile_geE1000r125_nbin10x',
            'profile_geE1000r125_exdiv10', 'profile_geE1000r125_exdiv2','profile_geE1000r125_ex5x',
           'profile_geE1000r125_exdiv5', 'profile_geE1000r125_nbin20x', 'profile_geE1000r125_nbin40x',
            'profile_geE1000r125_nbin100x']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-31000:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity



MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_geE1000r125'), np.loadtxt('Thermo_geE1000r125_nbin5x'), np.loadtxt('Thermo_geE1000r125_nbin10x'),
                       np.loadtxt('Thermo_geE1000r125_exdiv10'), np.loadtxt('Thermo_geE1000r125_exdiv2'),
                       np.loadtxt('Thermo_geE1000r125_ex5x'), np.loadtxt('Thermo_geE1000r125_exdiv5'),
                       np.loadtxt('Thermo_geE1000r125_nbin20x'), np.loadtxt('Thermo_geE1000r125_nbin40x'),
                       np.loadtxt('Thermo_geE1000r125_nbin100x')], dtype = object)

labels2 = ['Si(hot)-r125 550 nm 50x800','Si(hot)-r125 550 nm 50x4000','Si(hot)-r125 550 nm 50x8000','Si(hot)-r125 550 nm 5x800','Si(hot)-r125 550 nm 25x800',
           'Si(hot)-r125 550 nm 250x800','Si(hot)-r125 550 nm 10x800', 'Si(hot)-r125 550 nm 50x16000', 'Si(hot)-r125 550 nm 50x32000', 'Si(hot)-r125 550 nm 50x80000'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels2[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000],
                       Temperature[4000:4800],Temperature[4800:5600],Temperature[5600:6400],
                       Temperature[6400:7200],Temperature[7200:8000]], dtype=float) 
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600],Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000],
                      Coordinate[4000:4800],Coordinate[4800:5600],Coordinate[5600:6400],
                      Coordinate[6400:7200],Coordinate[7200:8000]], dtype=float) 




excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_Ge125_1000_xcr = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    
    average_temp = np.mean(dTemp)
    average_temp_Ge125_1000_xcr.append(average_temp)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
ThermalsGe125_1000_xcr = []
dt = 0.0005 # picoseconds

# Step Temp E_pair PotEng KinEng TotEng Temp Press Volume Lx Ly Lz f_3 v_tdiff
for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right2,Thermal_right1]
    Ttherm = ['%.6f' % elem for elem in Therm_val]
    ThermalsGe125_1000_xcr.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.6f' % elem for elem in TC_values]
    s_values.append(val)

therm_averageGe125_1000_xcr = []
for item in s_values:
    therm_averageGe125_1000_xcr.append(item[2])

therm_averageGe125_1000_xcr = []
for item in s_values:
    therm_averageGe125_1000_xcr.append(np.array(float(item[2])))

#projection stuff
sizeGe125_1000_xcr = [1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0]
sizeGe125_1000_xcr = np.array(sizeGe125_1000_xcr)
Tc = [float(s_values[0][2]), float(s_values[1][2]), float(s_values[2][2]), float(s_values[3][2]), float(s_values[4][2]),
     float(s_values[5][2]),float(s_values[6][2]),float(s_values[7][2]),float(s_values[8][2]),float(s_values[9][2])]

Tc = np.array(Tc)
inv_sizeGe125_1000_xcr = np.reciprocal(sizeGe125_1000_xcr)
inv_Tc = np.reciprocal(Tc)


SE_A1000b_left = np.std([float(ThermalsGe125_1000_xcr[0][1]), float(ThermalsGe125_1000_xcr[0][2])], dtype=np.float64)
SE_A1000_left = SE_A1000b_left/math.sqrt(2)
SE_A1000b_right = np.std([float(ThermalsGe125_1000_xcr[0][0]), float(ThermalsGe125_1000_xcr[0][3])], dtype=np.float64)
SE_A1000_right = SE_A1000b_right/math.sqrt(2)
standard_error_A1000b125 = math.sqrt(((SE_A1000_left/2)**2) + ((SE_A1000_right/2)**2))

SE_B1000b_left = np.std([float(ThermalsGe125_1000_xcr[1][1]), float(ThermalsGe125_1000_xcr[1][2])], dtype=np.float64)
SE_B1000_left = SE_B1000b_left/math.sqrt(2)
SE_B1000b_right = np.std([float(ThermalsGe125_1000_xcr[1][0]), float(ThermalsGe125_1000_xcr[1][3])], dtype=np.float64)
SE_B1000_right = SE_B1000b_right/math.sqrt(2)
standard_error_B1000b125 = math.sqrt(((SE_B1000_left/2)**2) + ((SE_B1000_right/2)**2))

SE_C1000b_left = np.std([float(ThermalsGe125_1000_xcr[2][1]), float(ThermalsGe125_1000_xcr[2][2])], dtype=np.float64)
SE_C1000_left = SE_C1000b_left/math.sqrt(2)
SE_C1000b_right = np.std([float(ThermalsGe125_1000_xcr[2][0]), float(ThermalsGe125_1000_xcr[2][3])], dtype=np.float64)
SE_C1000_right = SE_C1000b_right/math.sqrt(2)
standard_error_C1000b125 = math.sqrt(((SE_C1000_left/2)**2) + ((SE_C1000_right/2)**2))

SE_D1000b_left = np.std([float(ThermalsGe125_1000_xcr[3][1]), float(ThermalsGe125_1000_xcr[3][2])], dtype=np.float64)
SE_D1000_left = SE_D1000b_left/math.sqrt(2)
SE_D1000b_right = np.std([float(ThermalsGe125_1000_xcr[3][0]), float(ThermalsGe125_1000_xcr[3][3])], dtype=np.float64)
SE_D1000_right = SE_D1000b_right/math.sqrt(2)
standard_error_D1000b125 = math.sqrt(((SE_D1000_left/2)**2) + ((SE_D1000_right/2)**2))

SE_E1000b_left = np.std([float(ThermalsGe125_1000_xcr[4][1]), float(ThermalsGe125_1000_xcr[4][2])], dtype=np.float64)
SE_E1000_left = SE_E1000b_left/math.sqrt(2)
SE_E1000b_right = np.std([float(ThermalsGe125_1000_xcr[4][0]), float(ThermalsGe125_1000_xcr[4][3])], dtype=np.float64)
SE_E1000_right = SE_E1000b_right/math.sqrt(2)
standard_error_E1000b125 = math.sqrt(((SE_E1000_left/2)**2) + ((SE_E1000_right/2)**2))

SE_F1000b_left = np.std([float(ThermalsGe125_1000_xcr[5][1]), float(ThermalsGe125_1000_xcr[5][2])], dtype=np.float64)
SE_F1000_left = SE_F1000b_left/math.sqrt(2)
SE_F1000b_right = np.std([float(ThermalsGe125_1000_xcr[5][0]), float(ThermalsGe125_1000_xcr[5][3])], dtype=np.float64)
SE_F1000_right = SE_F1000b_right/math.sqrt(2)
standard_error_F1000b125 = math.sqrt(((SE_F1000_left/2)**2) + ((SE_F1000_right/2)**2))

SE_G1000b_left = np.std([float(ThermalsGe125_1000_xcr[6][1]), float(ThermalsGe125_1000_xcr[6][2])], dtype=np.float64)
SE_G1000_left = SE_G1000b_left/math.sqrt(2)
SE_G1000b_right = np.std([float(ThermalsGe125_1000_xcr[6][0]), float(ThermalsGe125_1000_xcr[6][3])], dtype=np.float64)
SE_G1000_right = SE_G1000b_right/math.sqrt(2)
standard_error_G1000b125 = math.sqrt(((SE_G1000_left/2)**2) + ((SE_G1000_right/2)**2))

SE_H1000b_left = np.std([float(ThermalsGe125_1000_xcr[7][1]), float(ThermalsGe125_1000_xcr[7][2])], dtype=np.float64)
SE_H1000_left = SE_H1000b_left/math.sqrt(2)
SE_H1000b_right = np.std([float(ThermalsGe125_1000_xcr[7][0]), float(ThermalsGe125_1000_xcr[7][3])], dtype=np.float64)
SE_H1000_right = SE_H1000b_right/math.sqrt(2)
standard_error_H1000b125 = math.sqrt(((SE_H1000_left/2)**2) + ((SE_H1000_right/2)**2))

SE_I1000b_left = np.std([float(ThermalsGe125_1000_xcr[8][1]), float(ThermalsGe125_1000_xcr[8][2])], dtype=np.float64)
SE_I1000_left = SE_I1000b_left/math.sqrt(2)
SE_I1000b_right = np.std([float(ThermalsGe125_1000_xcr[8][0]), float(ThermalsGe125_1000_xcr[8][3])], dtype=np.float64)
SE_I1000_right = SE_I1000b_right/math.sqrt(2)
standard_error_I1000b125 = math.sqrt(((SE_I1000_left/2)**2) + ((SE_I1000_right/2)**2))

SE_J1000b_left = np.std([float(ThermalsGe125_1000_xcr[9][1]), float(ThermalsGe125_1000_xcr[9][2])], dtype=np.float64)
SE_J1000_left = SE_J1000b_left/math.sqrt(2)
SE_J1000b_right = np.std([float(ThermalsGe125_1000_xcr[9][0]), float(ThermalsGe125_1000_xcr[9][3])], dtype=np.float64)
SE_J1000_right = SE_J1000b_right/math.sqrt(2)
standard_error_J1000b125 = math.sqrt(((SE_J1000_left/2)**2) + ((SE_J1000_right/2)**2))

yerrGe_A1000b125 = standard_error_A1000b125
yerrGe_B1000b125 = standard_error_B1000b125
yerrGe_C1000b125 = standard_error_C1000b125
yerrGe_D1000b125 = standard_error_D1000b125
yerrGe_E1000b125 = standard_error_E1000b125
yerrGe_F1000b125 = standard_error_F1000b125
yerrGe_G1000b125 = standard_error_G1000b125
yerrGe_H1000b125 = standard_error_H1000b125
yerrGe_I1000b125 = standard_error_I1000b125
yerrGe_J1000b125 = standard_error_J1000b125


In [ ]:
filename = ['./profile_siE800r271', 'profile_siE800r271_nbin5x', 'profile_siE800r271_nbin10x',
            'profile_siE800r271_10xbin_ex10div','profile_siE800r271_10xbin_ex2div', 'profile_siE800r271_5xbin_ex10div',
            'profile_siE800r271_5xbin_ex2div','profile_siE800r271_exdiv10','profile_siE800r271_exdiv2',
            'profile_siE800r271_ex5x','profile_siE800r271_exdiv5']


MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity



MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_siE800r271'), np.loadtxt('Thermo_siE800r271_nbin5x'), np.loadtxt('Thermo_siE800r271_nbin10x'),
                       np.loadtxt('Thermo_siE800r271_10xbin_ex10div'),np.loadtxt('Thermo_siE800r271_10xbin_ex2div'), np.loadtxt('Thermo_siE800r271_5xbin_ex10div'),
                        np.loadtxt('Thermo_siE800r271_5xbin_ex2div'),np.loadtxt('Thermo_siE800r271_exdiv10'),np.loadtxt('Thermo_siE800r271_exdiv2'),
                        np.loadtxt('Thermo_siE800r271_ex5x'), np.loadtxt('Thermo_siE800r271_exdiv5')], dtype = object)

labels3 = ['Ge(hot)-r271 439 nm 50x800','Ge(hot)-r271 439 nm 50x4000','Ge(hot)-r271 439 nm 50x8000',
           'Ge(hot)-r271 439 nm 5x8000','Ge(hot)-r271 439 nm 25x8000','Ge(hot)-r271 439 nm 5x4000',
           'Ge(hot)-r271 439 nm 25x4000','Ge(hot)-r271 439 nm 5x800','Ge(hot)-r271 439 nm 25x800',
           'Ge(hot)-r271 439 nm 250x800', 'Ge(hot)-r271 439 nm 10x800'
           ] 



for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]
 
fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels3[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000],Temperature[4000:4800],
                        Temperature[4800:5600],Temperature[5600:6400],Temperature[6400:7200],
                       Temperature[7200:8000],Temperature[8000:8800]], dtype=float)#, 
                       
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000],Coordinate[4000:4800],
                        Coordinate[4800:5600],Coordinate[5600:6400],Coordinate[6400:7200],
                      Coordinate[7200:8000],Coordinate[8000:8800]], dtype=float)#, 
                      


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_Si271 = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)

    average_temp = np.mean(dTemp)
    average_temp_Si271.append(average_temp)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
ThermalsSi271 = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
   
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    print(SE)
       
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right2,Thermal_right1]
    Ttherm = ['%.6f' % elem for elem in Therm_val]
    ThermalsSi271.append(Ttherm)

    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.6f' % elem for elem in TC_values]
    s_values.append(val)
    print('s values', labels3[simNo], ' = ', val)


    
therm_averageSi271 = []
for item in s_values:
    therm_averageSi271.append(item[2])

therm_averageSi271 = []
for item in s_values:
    therm_averageSi271.append(np.array(float(item[2])))

#projection stuff
sizeSi271 = [800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0]
sizeSi271 = np.array(sizeSi271)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),
      float(s_values[3][2]),float(s_values[4][2]),float(s_values[5][2]),
      float(s_values[6][2]),float(s_values[7][2]),float(s_values[8][2]),
     float(s_values[9][2]),float(s_values[10][2])]
      
Tc = np.array(Tc)

inv_sizeSi271 = np.reciprocal(sizeSi271)
inv_Tc = np.reciprocal(Tc)


SE_A800b_left = np.std([float(ThermalsSi271[0][1]), float(ThermalsSi271[0][2])], dtype=np.float64)
SE_A800_left = SE_A800b_left/math.sqrt(2)
SE_A800b_right = np.std([float(ThermalsSi271[0][0]), float(ThermalsSi271[0][3])], dtype=np.float64)
SE_A800_right = SE_A800b_right/math.sqrt(2)
standard_error_A800b271 = math.sqrt(((SE_A800_left/2)**2) + ((SE_A800_right/2)**2))

SE_B800b_left = np.std([float(ThermalsSi271[1][1]), float(ThermalsSi271[1][2])], dtype=np.float64)
SE_B800_left = SE_B800b_left/math.sqrt(2)
SE_B800b_right = np.std([float(ThermalsSi271[1][0]), float(ThermalsSi271[1][3])], dtype=np.float64)
SE_B800_right = SE_B800b_right/math.sqrt(2)
standard_error_B800b271 = math.sqrt(((SE_B800_left/2)**2) + ((SE_B800_right/2)**2))

SE_C800b_left = np.std([float(ThermalsSi271[2][1]), float(ThermalsSi271[2][2])], dtype=np.float64)
SE_C800_left = SE_C800b_left/math.sqrt(2)
SE_C800b_right = np.std([float(ThermalsSi271[2][0]), float(ThermalsSi271[2][3])], dtype=np.float64)
SE_C800_right = SE_C800b_right/math.sqrt(2)
standard_error_C800b271 = math.sqrt(((SE_C800_left/2)**2) + ((SE_C800_right/2)**2))

SE_D800b_left = np.std([float(ThermalsSi271[3][1]), float(ThermalsSi271[3][2])], dtype=np.float64)
SE_D800_left = SE_D800b_left/math.sqrt(2)
SE_D800b_right = np.std([float(ThermalsSi271[3][0]), float(ThermalsSi271[3][3])], dtype=np.float64)
SE_D800_right = SE_D800b_right/math.sqrt(2)
standard_error_D800b271 = math.sqrt(((SE_D800_left/2)**2) + ((SE_D800_right/2)**2))

SE_E800b_left = np.std([float(ThermalsSi271[4][1]), float(ThermalsSi271[4][2])], dtype=np.float64)
SE_E800_left = SE_E800b_left/math.sqrt(2)
SE_E800b_right = np.std([float(ThermalsSi271[4][0]), float(ThermalsSi271[4][3])], dtype=np.float64)
SE_E800_right = SE_E800b_right/math.sqrt(2)
standard_error_E800b271 = math.sqrt(((SE_E800_left/2)**2) + ((SE_E800_right/2)**2))

SE_F800b_left = np.std([float(ThermalsSi271[5][1]), float(ThermalsSi271[5][2])], dtype=np.float64)
SE_F800_left = SE_F800b_left/math.sqrt(2)
SE_F800b_right = np.std([float(ThermalsSi271[5][0]), float(ThermalsSi271[5][3])], dtype=np.float64)
SE_F800_right = SE_F800b_right/math.sqrt(2)
standard_error_F800b271 = math.sqrt(((SE_F800_left/2)**2) + ((SE_F800_right/2)**2))

SE_G800b_left = np.std([float(ThermalsSi271[6][1]), float(ThermalsSi271[6][2])], dtype=np.float64)
SE_G800_left = SE_G800b_left/math.sqrt(2)
SE_G800b_right = np.std([float(ThermalsSi271[6][0]), float(ThermalsSi271[6][3])], dtype=np.float64)
SE_G800_right = SE_G800b_right/math.sqrt(2)
standard_error_G800b271 = math.sqrt(((SE_G800_left/2)**2) + ((SE_G800_right/2)**2))

SE_H800b_left = np.std([float(ThermalsSi271[7][1]), float(ThermalsSi271[7][2])], dtype=np.float64)
SE_H800_left = SE_H800b_left/math.sqrt(2)
SE_H800b_right = np.std([float(ThermalsSi271[7][0]), float(ThermalsSi271[7][3])], dtype=np.float64)
SE_H800_right = SE_H800b_right/math.sqrt(2)
standard_error_H800b271 = math.sqrt(((SE_H800_left/2)**2) + ((SE_H800_right/2)**2))

SE_I800b_left = np.std([float(ThermalsSi271[8][1]), float(ThermalsSi271[8][2])], dtype=np.float64)
SE_I800_left = SE_I800b_left/math.sqrt(2)
SE_I800b_right = np.std([float(ThermalsSi271[8][0]), float(ThermalsSi271[8][3])], dtype=np.float64)
SE_I800_right = SE_I800b_right/math.sqrt(2)
standard_error_I800b271 = math.sqrt(((SE_I800_left/2)**2) + ((SE_I800_right/2)**2))

SE_J800b_left = np.std([float(ThermalsSi271[9][1]), float(ThermalsSi271[9][2])], dtype=np.float64)
SE_J800_left = SE_J800b_left/math.sqrt(2)
SE_J800b_right = np.std([float(ThermalsSi271[9][0]), float(ThermalsSi271[9][3])], dtype=np.float64)
SE_J800_right = SE_J800b_right/math.sqrt(2)
standard_error_J800b271 = math.sqrt(((SE_J800_left/2)**2) + ((SE_J800_right/2)**2))

SE_K800b_left = np.std([float(ThermalsSi271[10][1]), float(ThermalsSi271[10][2])], dtype=np.float64)
SE_K800_left = SE_K800b_left/math.sqrt(2)
SE_K800b_right = np.std([float(ThermalsSi271[10][0]), float(ThermalsSi271[10][3])], dtype=np.float64)
SE_K800_right = SE_K800b_right/math.sqrt(2)
standard_error_K800b271 = math.sqrt(((SE_K800_left/2)**2) + ((SE_K800_right/2)**2))


yerrSi_A800b271 = standard_error_A800b271
yerrSi_B800b271 = standard_error_B800b271
yerrSi_C800b271 = standard_error_C800b271
yerrSi_D800b271 = standard_error_D800b271
yerrSi_E800b271 = standard_error_E800b271
yerrSi_F800b271 = standard_error_F800b271
yerrSi_G800b271 = standard_error_G800b271
yerrSi_H800b271 = standard_error_H800b271
yerrSi_I800b271 = standard_error_I800b271
yerrSi_J800b271 = standard_error_J800b271
yerrSi_K800b271 = standard_error_K800b271


In [ ]:


sizeGe50_200_xcr = [110.0,110.0,110.0,110.0,110.0,110.0]

sizeGe125_1000_xcr = [550.0,550.0,550.0,550.0,550.0,550.0,550.0,550.0,550.0,550.0]

sizeSi271 = [439.9,439.9,439.9,439.9,439.9,439.9,439.9,439.9,439.9,439.9,439.9,]

In [ ]:

fig, ax = plt.subplots(figsize=(12,8))

for i in range(0,len(labels2)):
    #Si(hot)-r125 550 nm
    plt.plot(sizeGe125_1000_xcr[i], np.array(therm_averageGe125_1000_xcr[i]),'s-.', ms=8, label=labels2[i],markerfacecolor='none', markeredgecolor=color_blind_safe[i], color=color_blind_safe[i])
plt.plot(np.mean(sizeGe125_1000_xcr[:]), np.mean(np.array(therm_averageGe125_1000_xcr[:])),'s-.', ms=8, label='average',markerfacecolor='none', markeredgecolor='black', color='black')

plt.errorbar(sizeGe125_1000_xcr[0], therm_averageGe125_1000_xcr[0], yerrGe_A1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0], color=color_blind_safe[0])
plt.errorbar(sizeGe125_1000_xcr[1], therm_averageGe125_1000_xcr[1], yerrGe_B1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1])
plt.errorbar(sizeGe125_1000_xcr[2], therm_averageGe125_1000_xcr[2], yerrGe_C1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2])
plt.errorbar(sizeGe125_1000_xcr[3], therm_averageGe125_1000_xcr[3], yerrGe_D1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3], color=color_blind_safe[3])
plt.errorbar(sizeGe125_1000_xcr[4], therm_averageGe125_1000_xcr[4], yerrGe_E1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4], color=color_blind_safe[4])
plt.errorbar(sizeGe125_1000_xcr[5], therm_averageGe125_1000_xcr[5], yerrGe_F1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5], color=color_blind_safe[5])
plt.errorbar(sizeGe125_1000_xcr[6], therm_averageGe125_1000_xcr[6], yerrGe_G1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[6], color=color_blind_safe[6])
plt.errorbar(sizeGe125_1000_xcr[7], therm_averageGe125_1000_xcr[7], yerrGe_H1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7], color=color_blind_safe[7])
plt.errorbar(sizeGe125_1000_xcr[8], therm_averageGe125_1000_xcr[8], yerrGe_I1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[8], color=color_blind_safe[8])
plt.errorbar(sizeGe125_1000_xcr[9], therm_averageGe125_1000_xcr[9], yerrGe_J1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[9], color=color_blind_safe[9])

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.ylabel(r'$\kappa$ (W/m·K)', fontdict={'fontsize':27})

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 

plt.xlim(549,551)

plt.xticks(np.arange(549, 552, 1))

plt.show

In [ ]:

fig, ax = plt.subplots(figsize=(12,8))

for i in range(0,len(labels3)):
    #Ge(hot)-r271 439 nm
    plt.plot(sizeSi271[i], np.array(therm_averageSi271[i]),'s-.', ms=8, label=labels3[i],markerfacecolor='none', markeredgecolor=color_blind_safe[i], color=color_blind_safe[i])
plt.plot(np.mean(sizeSi271[:]), np.mean(np.array(therm_averageSi271[:])),'s-.', ms=8, label=labels3[i],markerfacecolor='none', markeredgecolor='black', color='black')

plt.errorbar(sizeSi271[0], therm_averageSi271[0], yerrSi_A800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0], color=color_blind_safe[0])
plt.errorbar(sizeSi271[1], therm_averageSi271[1], yerrSi_B800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1])
plt.errorbar(sizeSi271[2], therm_averageSi271[2], yerrSi_C800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2])
plt.errorbar(sizeSi271[3], therm_averageSi271[3], yerrSi_D800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3], color=color_blind_safe[3])
plt.errorbar(sizeSi271[4], therm_averageSi271[4], yerrSi_E800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4], color=color_blind_safe[4])
plt.errorbar(sizeSi271[5], therm_averageSi271[5], yerrSi_F800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5], color=color_blind_safe[5])
plt.errorbar(sizeSi271[6], therm_averageSi271[6], yerrSi_G800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[6], color=color_blind_safe[6])
plt.errorbar(sizeSi271[7], therm_averageSi271[7], yerrSi_H800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7], color=color_blind_safe[7])
plt.errorbar(sizeSi271[8], therm_averageSi271[8], yerrSi_I800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[8], color=color_blind_safe[8])
plt.errorbar(sizeSi271[9], therm_averageSi271[9], yerrSi_J800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[9], color=color_blind_safe[9])
plt.errorbar(sizeSi271[10], therm_averageSi271[10], yerrSi_K800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[10], color=color_blind_safe[10])


plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.ylabel(r'$\kappa$ (W/m·K)', fontdict={'fontsize':27})

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 

plt.xlim(438.9,440.9)

plt.xticks(np.arange(438.9, 441.9, 1))


plt.show

In [ ]:
print(sizeGe125_1000_xcr[0], np.array(therm_averageGe125_1000_xcr[:]))
print(sizeSi271[0], np.array(therm_averageSi271[:]))

In [ ]:

filename = ['./profile_geE1000r125', 'profile_geE1000r125_nbin5x', 'profile_geE1000r125_nbin10x',
            'profile_geE1000r125_exdiv10', 'profile_geE1000r125_exdiv2','profile_geE1000r125_ex5x',
           'profile_geE1000r125_exdiv5', 'profile_geE1000r125_nbin20x', 'profile_geE1000r125_nbin40x',
           'profile_geE1000r125_nbin100x']

MP_Profile = []; MP_Ave = []; #MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_geE1000r125'), np.loadtxt('Thermo_geE1000r125_nbin5x'), np.loadtxt('Thermo_geE1000r125_nbin10x'),
                       np.loadtxt('Thermo_geE1000r125_exdiv10'), np.loadtxt('Thermo_geE1000r125_exdiv2'),
                       np.loadtxt('Thermo_geE1000r125_ex5x'), np.loadtxt('Thermo_geE1000r125_exdiv5'),
                       np.loadtxt('Thermo_geE1000r125_nbin20x'),np.loadtxt('Thermo_geE1000r125_nbin40x'),
                       np.loadtxt('Thermo_geE1000r125_nbin100x')], dtype = object)

labels2 = ['Si(hot)-r125 550 nm 50x800','Si(hot)-r125 550 nm 50x4000','Si(hot)-r125 550 nm 50x8000','Si(hot)-r125 550 nm 5x800','Si(hot)-r125 550 nm 25x800',
           'Si(hot)-r125 550 nm 250x800','Si(hot)-r125 550 nm 10x800', 'Si(hot)-r125 550 nm 50x16000', 'Si(hot)-r125 550 nm 50x32000','Si(hot)-r125 550 nm 50x80000' ]


Temperature = []; Coordinate = []; dX = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000],Temperature[4000:4800],
                        Temperature[4800:5600],Temperature[5600:6400],Temperature[6400:7200],
                       Temperature[7200:8000]], dtype=float)#, 
                       
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000],Coordinate[4000:4800],
                        Coordinate[4800:5600],Coordinate[5600:6400],Coordinate[6400:7200],
                      Coordinate[7200:8000]], dtype=float)#, 
 

###
fig = plt.figure(figsize=(12,8))
for k in range(0, len(filename)):
    
    min_T = min(Temperature[k])
    max_T = max(Temperature[k])
    min_C = min(Coordinate[k])
    
    min_TT = np.full((800, ), min_T)
    min_CC = np.full((800, ), min_C)
    
    plt.plot(Coordinate[k]*0.1, Temperature[k], color=colors[k], label=labels2[k])
    #plt.plot(Coordinate[k]*0.1, min_TT, color=colors[k])

#plt.xlim(0,200)  
plt.xlabel('Position in Supercell (nm)')
plt.ylabel('Temperature (K)')
plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}


plt.show



In [ ]:

filename = ['./profile_siE800r271', 'profile_siE800r271_nbin5x', 'profile_siE800r271_nbin10x',
            'profile_siE800r271_10xbin_ex10div','profile_siE800r271_10xbin_ex2div', 'profile_siE800r271_5xbin_ex10div',
            'profile_siE800r271_5xbin_ex2div','profile_siE800r271_exdiv10','profile_siE800r271_exdiv2',
            'profile_siE800r271_ex5x','profile_siE800r271_exdiv5']

MP_Profile = []; MP_Ave = []; #MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity


Thermo_Data = np.array([np.loadtxt('./Thermo_siE800r271'), np.loadtxt('Thermo_siE800r271_nbin5x'), np.loadtxt('Thermo_siE800r271_nbin10x'),
                       np.loadtxt('Thermo_siE800r271_10xbin_ex10div'),np.loadtxt('Thermo_siE800r271_10xbin_ex2div'), np.loadtxt('Thermo_siE800r271_5xbin_ex10div'),
                        np.loadtxt('Thermo_siE800r271_5xbin_ex2div'),np.loadtxt('Thermo_siE800r271_exdiv10'),np.loadtxt('Thermo_siE800r271_exdiv2'),
                        np.loadtxt('Thermo_siE800r271_ex5x'), np.loadtxt('Thermo_siE800r271_exdiv5')], dtype = object)

labels3 = ['Ge(hot)-r271 439 nm 50x800','Ge(hot)-r271 439 nm 50x4000','Ge(hot)-r271 439 nm 50x8000',
           'Ge(hot)-r271 439 nm 5x8000','Ge(hot)-r271 439 nm 25x8000','Ge(hot)-r271 439 nm 5x4000',
           'Ge(hot)-r271 439 nm 25x4000','Ge(hot)-r271 439 nm 5x800','Ge(hot)-r271 439 nm 25x800',
           'Ge(hot)-r271 439 nm 250x800', 'Ge(hot)-r271 439 nm 10x8000']
 


Temperature = []; Coordinate = []; dX = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000],Temperature[4000:4800],
                        Temperature[4800:5600],Temperature[5600:6400],Temperature[6400:7200],
                       Temperature[7200:8000],Temperature[8000:8800]], dtype=float)#, 
                       
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000],Coordinate[4000:4800],
                        Coordinate[4800:5600],Coordinate[5600:6400],Coordinate[6400:7200],
                      Coordinate[7200:8000],Coordinate[8000:8800]], dtype=float)#, 
 
###
fig = plt.figure(figsize=(12,8))
for k in range(0, len(filename)):
    
    min_T = min(Temperature[k])
    max_T = max(Temperature[k])
    min_C = min(Coordinate[k])
    
    min_TT = np.full((800, ), min_T)
    min_CC = np.full((800, ), min_C)
    
    plt.plot(Coordinate[k]*0.1, Temperature[k], color=colors[k], label=labels3[k])
    #plt.plot(Coordinate[0]*0.1, Temperature[0], color=colors[0], label=labels3[0])
    #plt.plot(Coordinate[k]*0.1, min_TT, color=colors[k])

#plt.xlim(0,200)  
plt.xlabel('Position in Supercell (nm)')
plt.ylabel('Temperature (K)')
plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}

plt.show



In [ ]:
print(np.max(Temperature[2]),np.max(Temperature[0]))
print(np.max(Temperature[3]),np.max(Temperature[7]))

print(np.max(Temperature[2]) - np.max(Temperature[0]))
print(np.max(Temperature[3]) - np.max(Temperature[7]))

In [ ]:
print(np.max(therm_averageSi271[2]),np.max(therm_averageSi271[0]))
print(np.max(therm_averageSi271[3]),np.max(therm_averageSi271[7]))
print(np.max(therm_averageSi271[2]) - np.max(therm_averageSi271[0]))
print(np.max(therm_averageSi271[3]) - np.max(therm_averageSi271[7]))